# Preprocessing del dataset xView per il problema di object detection

## About this notebook
Questo notebook viene utilizzato per la preparazione del dataset. In particolare al termine di tutte le operazioni avremo una suddivisione dei dati in train-validation-test (labellati) + test (non labellato). Ci servirà per eseguire l'addestramento tramite reti YOLOv8 e ResNet.
Vengono estratte tutte le informazioni utili dal file xView_train.geojson e convertite in un formato adatto alle nostre reti (formato YOLO e COCO).
Dal momento che le nostre reti lavorano con dimensioni di input fissate, nel nostro caso 640x640, tutte le immagini, ad eccezione di quelle di test, vengono croppate per rispettare le dimensioni scelte. In questo modo la rete non esegue un resize automatico di conseguenza non rischiamo di perdere informazioni.
Dato che le immagini originali non sono tutte della stessa dimensione, potrebbe capitare che alcuni crop non ricadano nella dimensione scelta, per evitare questo problema abbiamo deciso anche di eseguire un padding sulle immagini di dimensione minore di 640x640.
Per quanto riguarda invece le immagini di test (sia labellato che non labellato) non sono state croppate perchè in questo modo siamo capaci di capire il comportamento della rete sull'immagine a dimensioni originali sulla quale vogliamo fare detection. Sarà poi compito della rete dividere l'immagine in crop, fare predizione su questi ultimi e mettere insieme i risultati relativi ad una stessa immagine.

### File structure
La cartella /kaggle/working conterrà il dataset che andrà in input alle reti per la detection (in altri 2 notebook).
In particolare avremo le seguenti directory:
- images: contiene tutte le immagini di train, validation e test. Quelle appartenenti a train e validation sono croppate come indicato sopra. *Es img_0_640.jpg*
- labels: per ogni immagine è presente un file di testo contenente tutti i relativi box. *Es img_0_640.txt*
- YOLO_cfg: contiene 4 files: <b>*test.txt, train.txt, val.txt*</b> che contengono i percorsi alle immagini rispettivamente di test, train e validation. <b>*xView_yolo.yaml*</b> che è il file di configurazione preso in input dalla rete YOLO per far partire l'addestramento.
- TestImages: contiene tutte le immagini di test (senza labels).

E i seguenti files:
- COCO_annotations.json: è la versione COCO del file di configurazione YOLO, che viene usato dalla rete ResNet.
- xView_class_map.json: associa ad ogni classe (string) il corrispettivo indice (intero) da 0 a 59.
- xview_labels.parquet: contiene tutti i box delle immagini in forma tabellata.

In [ ]:
shutil.rmtree('/kaggle/working/labels')
shutil.rmtree('/kaggle/working/YOLO_cfg')
shutil.rmtree('/kaggle/working/images')
os.remove('/kaggle/working/xView_class_map.json')

In [ ]:
from pathlib import Path
import pandas as pd
import cv2
import os
import numpy as np
from os import sep
import shutil
import json
import yaml
import matplotlib.pyplot as plt
import random
import time
from tqdm.notebook import tqdm_notebook
import concurrent.futures
import multiprocessing as mp
from PIL import Image
import zipfile
from IPython.display import FileLink

### Setup
Definizione di tutti i percorsi utili in questo notebook.

In [ ]:
#Data sources
DATA_FLDR_NM = 'Data'
IN_DATASET_NM = 'xview-dataset'
IMAGE_FLDR_NM = 'train_images'
IN_LABELS_FLDR_NM = 'train_labels'
LABELS_XML_NM = 'xView_train.geojson'

#Output folders and file names
OUT_DATASET_NM = 'xview-dataset-team1/xview-dataset-team1'
CLASS_MAP_JSON_NM = 'xView_class_map.json'
OUT_COCO_JSON_NM = 'COCO_annotations.json'
OUT_IMAGE_FLDR_NM = 'images'
OUT_LABELS_FLDR_NM = 'labels'
OUT_CFG_FLDR_NM = 'YOLO_cfg'
OUT_DATAFRAME_NM = 'xview_labels.parquet'
YAML_NM = 'xview_yolo.yaml'
CHUNK_WIDTH = 640  # width of the images being created
CHUNK_HEIGHT = 640
MIN_CHUNK_HEIGHT = 160 # no images will be kept if the image chunk is smaller than this
MIN_CHUNK_WIDTH = 160
IMAGE_WRITING = True #True to re-perform image cropping, False just to regenerated other data
TEST_FRACTION = 0.1
JPEG_COMPRESSION = 95 # For the saved files
VAL_FRACTION = 0.1
RANDOM_SEED = 2023
DEBUG = False


in_dataset_pth = Path('/kaggle/input/xview-dataset')
out_dataset_pth = Path('/kaggle/working/')
future_ds_img_fldr = Path(f'/kaggle/input/{OUT_DATASET_NM}/{OUT_IMAGE_FLDR_NM}') #Questo path serve per scrivere nei file .txt il percorso corretto alle immagini del dataset che esportiamo
future_ds_cfg_fldr = Path(f'/kaggle/input/{OUT_DATASET_NM}/{OUT_CFG_FLDR_NM}')


labels_json_pth = in_dataset_pth / IN_LABELS_FLDR_NM / LABELS_XML_NM
img_fldr_pth = in_dataset_pth / IMAGE_FLDR_NM / IMAGE_FLDR_NM
save_images_fldr_pth = out_dataset_pth / OUT_IMAGE_FLDR_NM 
save_labels_fldr_pth = out_dataset_pth / OUT_LABELS_FLDR_NM
out_data_parquet_pth = out_dataset_pth / OUT_DATAFRAME_NM
out_json_map_pth = out_dataset_pth / CLASS_MAP_JSON_NM 
class_map_pth = out_dataset_pth / CLASS_MAP_JSON_NM
cfg_fldr_pth = out_dataset_pth / OUT_CFG_FLDR_NM
coco_json_pth = out_dataset_pth / OUT_COCO_JSON_NM
yolo_yaml_pth = cfg_fldr_pth / YAML_NM
train_txt_pth = cfg_fldr_pth / 'train.txt'
val_txt_pth = cfg_fldr_pth / 'train.txt'
test_txt_pth = cfg_fldr_pth / 'test.txt'


def make_empty_dir(directory):
    if directory.is_dir():
        shutil.rmtree(directory)
    os.makedirs(directory)

make_empty_dir(cfg_fldr_pth)
if IMAGE_WRITING:
    make_empty_dir(save_images_fldr_pth)
    make_empty_dir(save_labels_fldr_pth)

random.seed(RANDOM_SEED)

print(f'The input images are found at {cfg_fldr_pth}')
print(f'The input labels are found at  {labels_json_pth}')
print(f'Configuration files will be saved to {cfg_fldr_pth}')
print(f'YOLO image files will be saved to {save_images_fldr_pth}')
print(f'YOLO labels files will be saved to {save_labels_fldr_pth}')

## Helper Functions

In [ ]:
def convert_tif_to_jpg(input_path, output_path):
    # Open the .tif image
    with Image.open(input_path) as img:
        # Save the image in .png format
        img.save(output_path, format='JPEG')

        
#Questa funzione mi ritorna un dizionario contenente i box appartenenti a ciascun immagine (eventualmente si può limitare il numero di classi da considerare, grazie a class_lst)
def get_boxes(in_df, class_lst=[]):
    if class_lst:
        in_df = in_df[in_df['TYPE_ID'].isin(class_lst)]
    unique_images = in_df.IMAGE_ID.unique().tolist() #serve per scorrere le immagini presenti nel df
    boxs = {}

    for image in tqdm_notebook(unique_images):
        mask = in_df['IMAGE_ID'] == image #crea una maschera con valori 1 nelle locazioni in cui i box appartengono all'immagine corrente (image).
        masked = in_df[mask][['TYPE_ID', 'XMIN', 'YMIN', 'XMAX', 'YMAX']] #accedo alle righe del df corrispondenti agli 1 della maschera.
        boxs[image] = masked.values.tolist()
    return boxs


def load_bgr_image(file_pth):
    image_obj = cv2.imread(file_pth)
    return image_obj


#Adatta la lista di box in ingresso ai bordi dell'immagine definiti in chunk limits e converte le coordinate in formato yolo.
def match_boxes(box_list, chnk_lims):
    boxes_lists = []
    le, to = chnk_lims[0], chnk_lims[1]  # chunk_limits = [c, r, chunk_w, chunk_h]
    w, h  = chnk_lims[2], chnk_lims[3]
    for box in box_list:
        o_left, o_top, o_right, o_bottom = box[1], box[2], box[3], box[4]
        left, right = (o_left - le)/w, (o_right - le)/w  # translate and normalise
        top, bottom = (o_top - to)/h, (o_bottom - to)/h

        h_match = (0 <= left < 1) or (0 < right <= 1)
        v_match = (0 <= top < 1) or (0 < bottom <= 1)

        if v_match and h_match:
            clipped = np.clip([left, top, right, bottom], a_min=0, a_max=1) #Dato che v_match e h_match possono entrambe essere verificate se anche solo uno tra (left, right) e (top, bottom) sono comprese tra 0 e 1
                                                                            #quindi nel caso in cui 0<left<1 == true e 0<right<1 == false, dobbiamo mettere right a 1 in modo tale che il box non esca dal crop.
            l, t, r, b = clipped[0], clipped[1], clipped[2], clipped[3]
            bounding_box = [str(box[0]),
                            str(round((l + r)/2, 5)),
                            str(round((t + b)/2, 5)),
                            str(round(r-l, 5)),
                            str(round(b-t, 5))]
            boxes_lists.append(bounding_box)
    return boxes_lists


def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

## Data Extraction and Cleaning

Estrazione delle sole colonne di interesse.

In [ ]:
with open(labels_json_pth, 'r') as infile:
    data = json.load(infile)
    keys = list(data.keys())

feature_list = data['features']
COLUMNS = ['IMAGE_ID', 'TYPE_ID', 'XMIN', 'YMIN', 'XMAX', 'YMAX']

data = []
for feature in tqdm_notebook(feature_list):
    properties = feature['properties'] # a dict
    img_id = properties['image_id']  # '389.tif'
    type_id = properties['type_id']
    bbox = properties['bounds_imcoords'].split(",")  # eg '1917,38,1958,64'
    one_row = [img_id, type_id, bbox[0], bbox[1], bbox[2], bbox[3]]
    data.append(one_row)

instances = len(data)
print(f'There are {instances} object instances in the original dataset')

Conversione delle colonne contenenti numeri in interi.

In [ ]:
df = pd.DataFrame(data, columns = COLUMNS)
df[['XMIN', 'YMIN', 'XMAX', 'YMAX']] = df[['XMIN', 'YMIN', 'XMAX', 'YMAX']].apply(pd.to_numeric)
df.head()

Rimozione di annotazioni errate.

In [ ]:
df = df[(df.TYPE_ID != 75) & (df.TYPE_ID != 82)]   # removing erroneous labels
print(f'{instances - len(df)} rows removed, leaving {len(df)} rows')

In [ ]:
df.head()

Rimozione dell'immagine 1395 che non esiste nel dataset

In [ ]:
old_length = len(df)
df = df[df.IMAGE_ID != '1395.tif']
print(f'{old_length - len(df)} rows removed, leaving {len(df)}')
df.head()

Rimozione di tutti i box che hanno area nulla. (perchè hanno almeno una dimensione nulla)

In [ ]:
old_length = len(df)
df = df[df.XMIN != df.XMAX]
df = df[df.YMIN != df.YMAX]
print(f'{old_length - len(df)} rows removed, leaving {len(df)}')
df.head()

Vengono mappate le classi su un insieme ordinato di indici che va da 0 a 59.

In [ ]:
old_dict = {
    11:'Fixed-wing Aircraft', 12:'Small Aircraft', 13:'Passenger/Cargo Plane', 15:'Helicopter',
    17:'Passenger Vehicle', 18:'Small Car', 19:'Bus', 20:'Pickup Truck', 21:'Utility Truck',
    23:'Truck', 24:'Cargo Truck', 25:'Truck Tractor w/ Box Trailer', 26:'Truck Tractor',27:'Trailer',
    28:'Truck Tractor w/ Flatbed Trailer', 29:'Truck Tractor w/ Liquid Tank', 32:'Crane Truck',
    33:'Railway Vehicle', 34:'Passenger Car', 35:'Cargo/Container Car', 36:'Flat Car', 37:'Tank car',
    38:'Locomotive', 40:'Maritime Vessel', 41:'Motorboat', 42:'Sailboat', 44:'Tugboat', 45:'Barge',
    47:'Fishing Vessel', 49:'Ferry', 50:'Yacht', 51:'Container Ship', 52:'Oil Tanker',
    53:'Engineering Vehicle', 54:'Tower crane', 55:'Container Crane', 56:'Reach Stacker',
    57:'Straddle Carrier', 59:'Mobile Crane', 60:'Dump Truck', 61:'Haul Truck', 62:'Scraper/Tractor',
    63:'Front loader/Bulldozer', 64:'Excavator', 65:'Cement Mixer', 66:'Ground Grader', 71:'Hut/Tent',
    72:'Shed', 73:'Building', 74:'Aircraft Hangar', 76:'Damaged Building', 77:'Facility', 79:'Construction Site',
    83:'Vehicle Lot', 84:'Helipad', 86:'Storage Tank', 89:'Shipping container lot', 91:'Shipping Container',
    93:'Pylon', 94:'Tower'}


old_keys = sorted(list(old_dict.keys()))
new_dict = {old_dict[x]:y for y, x in enumerate(old_keys)}
class_map_dict = {y:old_dict[x] for y, x in enumerate(old_keys)}
with open(out_json_map_pth, "w") as json_file:
    json.dump(class_map_dict, json_file)
print(class_map_dict)

df['TYPE_ID'] = df['TYPE_ID'].apply(lambda x: new_dict[old_dict[x]])
df.head(3)

## Main Process
- Split in train-test-validation.
- Divisione delle immagini in crop.
- Salvataggio dei crop in formato jpg nella cartella /kaggle/working/images.
- Conversione delle labels in formato YOLO: x_center, y_center, width, height (tutte normalizzate).

In [ ]:
boxes_dict = get_boxes(df) #restituisce un dizionario in forma {filename:[['TYPE_ID', 'XMIN', 'YMIN', 'XMAX', 'YMAX'],[..],[..],..]}

In [ ]:
def process_image(img_fn, 
                  dir_pth=img_fldr_pth, 
                  boxes=boxes_dict, 
                  out_dir=save_images_fldr_pth, 
                  c_height=CHUNK_HEIGHT, 
                  c_width=CHUNK_WIDTH,  
                  jpg_q=JPEG_COMPRESSION,
                  min_h=MIN_CHUNK_HEIGHT,
                  min_w=MIN_CHUNK_WIDTH,
                  writing=IMAGE_WRITING
                 ):
    
    labels_list = boxes[img_fn]
    img_pth = str(dir_pth / img_fn)
    im = load_bgr_image(img_pth)
    full_h, full_w, _ = im.shape
    y_boxes= {}
    f_names, widths, heights = [], [], []
    
    for r in range(0, full_h, c_height):
        for c in range(0, full_w, c_width):
            stem = img_fn.split('.')[0]
            fn = str(f"img_{stem}_{r}_{c}.jpg")
            out_pth = str(out_dir / fn)
            width = c_width
            height = c_height
            if r + height > full_h:
                height = full_h - r
            if c + width > full_w:
                width = full_w - c
            big_enough = (height > min_h) and (width > min_w) #non consideriamo i crop con dimensione inferiore a min_h x min_w (160x160)
            if big_enough:
                if (height == 640) and (width == 640):  
                    if writing:
                        cv2.imwrite(out_pth, im[r:r+height, c:c+width,:],  [int(cv2.IMWRITE_JPEG_QUALITY), jpg_q])
                        
                else:        #Facciamo padding all'immagine per avere tutti i crop di dimensione 640x640
                    temp = im[r:r+height, c:c+width, :]
                    if width < 640:
                        padding = np.zeros((height, 640-width, 3), dtype=np.uint8)
                        temp = np.concatenate((temp,padding), axis=1)     #concatena l'immagine temp e gli zeri lungo la direzione orizzontale
                        width = 640
                        
                    if height < 640:
                        padding = np.zeros((640-height, width, 3), dtype=np.uint8)
                        temp = np.concatenate((temp,padding), axis=0)     #concatena l'immagine temp e gli zeri lungo la direzione verticale
                        height = 640
                        
                    if writing:
                        cv2.imwrite(out_pth, temp,  [int(cv2.IMWRITE_JPEG_QUALITY), jpg_q])
                
                    if(temp.shape != (640,640,3)):
                        print('errore')
                    
                    
                chunk_limits = [c, r, width, height]
                y_boxes[fn] = match_boxes(labels_list, chunk_limits) #match boxes normalizza le coordinate dei box e le trasforma in [id, centrox, centroy, width, height]
                f_names.append(fn) #lista contenente i nomi di tutti i crop dell'immagine
                widths.append(width) #lista contenente le larghezze di tutti i crop dell'immagine
                heights.append(height) #lista contenente le altezze di tutti i crop dell'immagine
    return f_names, widths, heights, y_boxes

In [ ]:
img_fns = df.IMAGE_ID.unique().tolist() #ci prendiamo la lista di tutte le immagini (senza ripetizioni)

In [ ]:
filenames = [x for x in os.listdir('/kaggle/input/xview-dataset/train_images/train_images') if x.endswith(".tif")]

total_images = len(filenames)
indices = list(range(total_images))
random.shuffle(indices)

train_fraction = 1 - TEST_FRACTION - VAL_FRACTION
train_sp = int(np.floor(train_fraction * len(indices))) #training-validation split
valid_sp = int(np.floor(VAL_FRACTION * len(indices))) + train_sp #validation-test split
train_idx, val_idx, test_idx = indices[:train_sp], indices[train_sp:valid_sp], indices[valid_sp:]

print(' Training set size: \t', len(train_idx))
print(' Validation set size: \t', len(val_idx))
print(' Test set size: \t', len(test_idx))
print(' Total dataset: \t', total_images)

In [ ]:
files = ['train.txt', 'val.txt', 'test.txt']
splits = [train_idx, val_idx, test_idx]

for fn, split in zip(files, splits):
    txt_pth = cfg_fldr_pth / fn
    with open(txt_pth, 'a') as f:
        for ind in split:
            f.write(str(future_ds_img_fldr / filenames[ind]) + '\n')
        print(f'{fn[:-4]} file written to {txt_pth}, with {len(split) } samples')

Separazione delle immagini di train e validation da quelle di test in quanto solamente sulle prime vogliamo eseguire i crop.

In [ ]:
dataframe = df
print(len(df))
      
with open('/kaggle/working/YOLO_cfg/test.txt', 'r') as file:
    lines = [line.strip().split('/')[-1] for line in file.readlines()]

df2 = df[df['IMAGE_ID'].isin(lines)] #df2 contiene tutti i box delle sole immagini di test (presenti in test.txt)
df = df[~df['IMAGE_ID'].isin(lines)] #df contiene tutti i box delle immagini di train + validation

print(len(df))

img_fns = df.IMAGE_ID.unique().tolist() #img fns contiene tutti i box delle immagini di train + validation
img_fns2 = df2.IMAGE_ID.unique().tolist() #img fns2 contiene tutti i box delle immagini di test (locale)


Verifica che le immagini di test siano state correttamente separate da quelle di train e val.

In [ ]:
print(len(img_fns))
print(len(img_fns2))

Esecuzione della funzione process_image che realizza i crop.

In [ ]:
start_time = time.time()
num_threads = mp.cpu_count() 
overall_progress = tqdm_notebook(total=len(img_fns), desc="Creating and saving image tiles")
yolo_boxes= {}
file_names, widths, heights = [], [], []

with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    for f_names, c_widths, c_heights, y_boxes in executor.map(process_image, img_fns): #qui eseguiamo i crop per tutte le immagini di train + val
        file_names.extend(f_names)
        widths.extend(c_widths)
        heights.extend(c_heights)
        yolo_boxes.update(y_boxes)
        overall_progress.update(1)
overall_progress.close()

image_data = {file_names[i]: [widths[i], heights[i]] for i in range(len(file_names))}
time_taken=time.time() - start_time

Conversione e copia delle immagini indicate in test.txt dal dataset di input alla cartella /kaggle/working/images

In [ ]:
print(len(os.listdir('/kaggle/working/images')))

with open('/kaggle/working/YOLO_cfg/test.txt') as file:
    lines = [line.strip() for line in file] #ottengo il percorso dell'immagine

for line in lines:
    img_name = line.split('/')[-1]
    img_name_jpg = 'img_' + img_name.split('.')[0] + '.jpg'
    convert_tif_to_jpg('/kaggle/input/xview-dataset/train_images/train_images/' + img_name, '/kaggle/working/images/' + img_name_jpg)

print(len(os.listdir('/kaggle/working/images')))    

Per tutte le immagini contenute in lines (righe di test.txt) convertiamo le labels al formato YOLO

In [ ]:
print(len(yolo_boxes))
for line in lines:
    img_name_tif = line.strip().split('/')[-1]
    img_name_jpg = 'img_' + img_name_tif.split('.')[0] + '.jpg'
    img = plt.imread('/kaggle/working/images/' + img_name_jpg)
    
    height, width, _ = img.shape
   
    chunk_limits = [0,0,width, height]
    label_list = boxes_dict[img_name_tif]
    
    yolo_boxes[img_name_jpg] = match_boxes(label_list, chunk_limits)
    file_names.append(img_name_jpg)
    widths.append(width)
    heights.append(height)
print(len(yolo_boxes))

A partire da yolo_boxes, dizionario che contiene i box di tutte le immagini, creo un file di testo per ogni immagine contenente le labels nel formato [id, centro_x, centro_y, width, height]

In [ ]:
cont = 0
all_image_files = os.listdir(save_images_fldr_pth)
for image_fn in tqdm_notebook(all_image_files):
    stem = image_fn.split('.')[0]
    fn = str (stem) + '.txt'
    txt_pth = str(save_labels_fldr_pth / fn)
    seperator = ' '
    with open(txt_pth, 'a') as f:
        if image_fn in yolo_boxes:
            cont += 1
            for bbox in yolo_boxes[image_fn]:
                txt = seperator.join(bbox) + '\n'
                f.write(txt)

Creazione di un dataframe contenente tutti i box. Utile successivamente.

In [ ]:
text_paths = [save_labels_fldr_pth / x for x in os.listdir(save_labels_fldr_pth) if x.endswith(".txt")]
column_names = ['Class_ID', 'x_center', 'y_center', 'width', 'height']
data = []
for file_path in text_paths:
    with open(file_path, 'r') as file:
        for line in file:
            values = line.strip().split(' ')
            row_data = {col: val for col, val in zip(column_names, values)}
            row_data['File_Name'] = file_path.name
            data.append(row_data)

out_data = pd.DataFrame(data)
out_data['Class_ID']=out_df['Class_ID'].astype(int)
out_data['Class_Name'] = out_df['Class_ID'].map(class_map_dict).fillna('unknown')
out_data = out_df[['File_Name', 'Class_Name', 'Class_ID', 'x_center', 'y_center', 'width', 'height']]

out_data.head()


## Creazione del file in formato YOLO

In [ ]:
config = {'train': str(future_ds_cfg_fldr / files[0]),
          'val': str(future_ds_cfg_fldr / files[1]),
          'test': str(future_ds_cfg_fldr / files[2]),
          'nc': len(class_map_dict),
          'names': class_map_dict
          }

with open(yolo_yaml_pth, "w") as file:
    yaml.dump(config, file, default_style=None, default_flow_style=False, sort_keys=False)
print(f'yaml file written to {yolo_yaml_pth}')

## YOLO to COCO
Conversione del file dal formato YOLO al formato COCO

images:  
`{"id": int, "width": int, "height": int, "file_name": str, }`   
annotations:  
`{"id": int, "image_id": int, "category_id": int, "area": float, "bbox": [x,y,width,height]}`  
categories:  
`[{"id": int, "name": str}]`

Estrazione delle informazioni necessarie per la sezione "images" del file COCO.

In [ ]:
image_data = {'width': widths, 'height' : heights, 'file_name':file_names}
im_df = pd.DataFrame(image_data)
im_df['id'] = im_df['file_name'].str.replace(r'\D', '', regex=True).astype(int)
im_df.head()

In [ ]:
def row_to_dict(row):
    return {
        'id': row['id'],
        'width': row['width'],
        'height':row['height'],
        'file_name':row['file_name']
    }

im_list = im_df.apply(lambda row: row_to_dict(row), axis=1).tolist()
[print(val) for val in im_list[:4]]

Estrazione e conversione delle informazioni per la sezione "annotations" del file COCO.

In [ ]:
annotations_df = out_data.copy()
annotations_df['image_id'] = annotations_df['File_Name'].str.replace(r'\D', '', regex=True).astype(int)
annotations_df= annotations_df.rename(columns={'height': 'h', 'width': 'w'})
an_df = annotations_df.merge(im_df, left_on='image_id', right_on='id', how='left')
an_df['x_center']= (an_df['x_center'].astype(np.float64)*an_df['width']).round(decimals=0)
an_df['y_center']= (an_df['y_center'].astype(np.float64)*an_df['height']).round(decimals=0)
an_df['w']= (an_df['w'].astype(np.float64)*an_df['width']).round(decimals=0)
an_df['h']= (an_df['h'].astype(np.float64)*an_df['height']).round(decimals=0)
an_df['Class_ID']= an_df['Class_ID'].astype(int)
an_df = an_df.drop(columns=['File_Name', 'file_name', 'width', 'height', 'id'])
an_df['left'] = (an_df['x_center'] - an_df['w']/2).round(decimals=0)
an_df['top'] =  (an_df['y_center'] - an_df['h']/2).round(decimals=0)
an_df['bbox'] = ('[' + an_df['left'].astype(str) + ', ' 
              + an_df['top'].astype(str) + ', ' 
              + an_df['w'].astype(str) + ', '
              + an_df['h'].astype(str) + ']')
an_df['area'] = an_df['w'] * an_df['h']
an_df = an_df.drop(columns=['x_center', 'y_center', 'w', 'h', 'left', 'top', 'Class_Name'])
an_df.reset_index(inplace=True)
an_df.rename(columns={'index': 'id'}, inplace=True)
an_df.head()

In [ ]:
def row_to_dict(row):
    return {
        'id': row['id'],
        'image_id' : row['image_id'],
        'category_id': row['Class_ID'],
        'area':row['area'],
        'bbox':row['bbox']
    }

an_list = an_df.apply(lambda row: row_to_dict(row), axis=1).tolist()
print(an_list[:4])


Estrazione delle informazioni per la sezione "categories" del file COCO.

In [ ]:
cat_list = [{"id": int(key), "name": val} for key, val in class_map_dict.items()]
print(cat_list[:4])

Creazione e salvataggio del file COCO_annotations `.json`

In [ ]:
out_json_data = {'images': im_list, 'annotations': an_list, 'categories': cat_list}
with open(coco_json_pth, 'w') as json_file:
    json.dump(out_json_data, json_file, indent=4)
    
for key, value in out_json_data.items():
    print(key, value[:5])

Modifichiamo i file train.txt, val.txt, test.txt per fare in modo che contengano tutti i crop associati alle singole immagini.  
Al momento nel file .txt è contenuto il percorso all'immagine a dimensione originale ma non al crop.  
Vengono creati i nuovi file train2, val2, test2 da sostituire ai file originali

In [ ]:
#os.remove('/kaggle/working/YOLO_cfg/val2.txt')
#os.remove('/kaggle/working/YOLO_cfg/train2.txt')
#os.remove('/kaggle/working/YOLO_cfg/test2.txt')

for txt in ['train.txt', 'val.txt', 'test.txt']:
    filepath = '/kaggle/working/YOLO_cfg/' + txt
    with open(filepath, 'r') as file:
        lines = [line.strip() for line in file.readlines()]

    line = []
    for elem in lines:
        line.append(elem.split('/')[-1].split('.')[0])

    newfilepath = '/kaggle/working/YOLO_cfg/' + txt.split('.')[0] + '2.txt'
    with open(newfilepath, 'a') as file:
        for img in os.listdir('/kaggle/working/images/'):
            if (txt != 'test.txt'):
                if ((img.split('img_')[-1].split('_')[0]) in line): #sto prendendo solamente il numero associato all'immagine in 'images'
                    file.write(str(future_ds_img_fldr) + '/' + img + '\n')
            else:
                if ((img.split('img_')[-1].split('.')[0]) in line): #sto prendendo solamente il numero associato all'immagine in 'images'
                    file.write(str(future_ds_img_fldr) + '/' + img + '\n')


Eliminiamo i file train.txt, val.txt, test.txt e rinominiamo quelli creati al blocco di sopra per sostituirli.

In [ ]:
os.remove('/kaggle/working/YOLO_cfg/train.txt')
os.rename('/kaggle/working/YOLO_cfg/train2.txt', '/kaggle/working/YOLO_cfg/train.txt')

os.remove('/kaggle/working/YOLO_cfg/val.txt')
os.rename('/kaggle/working/YOLO_cfg/val2.txt', '/kaggle/working/YOLO_cfg/val.txt')

os.remove('/kaggle/working/YOLO_cfg/test.txt')
os.rename('/kaggle/working/YOLO_cfg/test2.txt', '/kaggle/working/YOLO_cfg/test.txt')

### Eseguo la conversione delle immagini di test (non labellate) da .tif a .jpg per averle nel dataset di output

In [ ]:
if not os.path.exists('/kaggle/working/TestImages'):
    os.mkdir('/kaggle/working/TestImages')

In [ ]:
from PIL import Image
for img_name in os.listdir('/kaggle/input/xview-dataset/val_images/val_images/'):
    img_path = '/kaggle/input/xview-dataset/val_images/val_images/' + img_name
    img_name = img_name.split('.')[0] + '.jpg'
    output_path = '/kaggle/working/TestImages/' + img_name
    convert_tif_to_jpg(img_path, output_path)

In [ ]:
os.listdir('/kaggle/working/')

# Download dataset

In [ ]:
zip_dir()